In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Fill in normalization factor
x_test = x_test / 255     # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),        # Fill input shape
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # Fill number of hidden neurons
    tf.keras.layers.Dense(10, activation='softmax')  # Fill number of output neurons
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',       # Fill name of loss function
              metrics=['accuracy'])

start = time.time()
model.fit(x_train, y_train, epochs=5)
end = time.time()
print(f"TF Training time: {end-start:.2f} seconds")       # Output training time
model.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8575 - loss: 0.5034
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9525 - loss: 0.1652
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9658 - loss: 0.1154
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9747 - loss: 0.0876
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9774 - loss: 0.0714
TF Training time: 15.40 seconds
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9677 - loss: 0.1104


[0.0948181003332138, 0.9715999960899353]

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\PARIGI~1\AppData\Local\Temp\tmpl2ooo4ye\assets


INFO:tensorflow:Assets written to: C:\Users\PARIGI~1\AppData\Local\Temp\tmpl2ooo4ye\assets


Saved artifact at 'C:\Users\PARIGI~1\AppData\Local\Temp\tmpl2ooo4ye'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  2959000558032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2959000558992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2959000558416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2959000558800: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [4]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [6]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transform: Convert to tensor and flatten 28x28 -> 784
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))  # Flatten
])

# Loaders
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

# Define Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)   # Input size 784 (28x28), hidden layer with 64 units
        self.fc2 = nn.Linear(64, 10)    # Hidden to 10 output classes

    def forward(self, x):
        x = F.relu(self.fc1(x))         # First layer + ReLU
        return self.fc2(x)              # Output layer (raw logits)

# Instantiate model
model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

# Training loop
start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

# Evaluation
model.eval()
correct = 0
with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()

print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")

PyTorch Training time: 65.69 seconds
Test accuracy: 0.9661


In [7]:
!pip install onnx

In [9]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

In [10]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalization factor: scale to [0, 1]
x_test = x_test / 255.0     # Same here
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Same batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),           # MNIST images are 28x28
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')  # Output layer with 10 neurons and softmax
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")


Epoch 1/5
Step 0, Loss: 2.4941, Accuracy: 0.0938
Step 100, Loss: 0.6565, Accuracy: 0.7163
Step 200, Loss: 0.2605, Accuracy: 0.7951
Step 300, Loss: 0.3571, Accuracy: 0.8293
Step 400, Loss: 0.3814, Accuracy: 0.8489
Step 500, Loss: 0.2175, Accuracy: 0.8623
Step 600, Loss: 0.1476, Accuracy: 0.8707
Step 700, Loss: 0.1284, Accuracy: 0.8777
Step 800, Loss: 0.2657, Accuracy: 0.8838
Step 900, Loss: 0.1583, Accuracy: 0.8881
Step 1000, Loss: 0.1333, Accuracy: 0.8923
Step 1100, Loss: 0.2524, Accuracy: 0.8963
Step 1200, Loss: 0.2566, Accuracy: 0.8995
Step 1300, Loss: 0.1443, Accuracy: 0.9028
Step 1400, Loss: 0.2683, Accuracy: 0.9052
Step 1500, Loss: 0.0969, Accuracy: 0.9074
Step 1600, Loss: 0.1453, Accuracy: 0.9098
Step 1700, Loss: 0.0367, Accuracy: 0.9123
Step 1800, Loss: 0.1156, Accuracy: 0.9143
Training Accuracy for epoch 1: 0.9159

Epoch 2/5
Step 0, Loss: 0.1746, Accuracy: 0.9062
Step 100, Loss: 0.0886, Accuracy: 0.9514
Step 200, Loss: 0.1643, Accuracy: 0.9518
Step 300, Loss: 0.0844, Accuracy:

In [11]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0   # Normalize to [0, 1]
x_test = x_test / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),         # MNIST images are 28x28
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),  # Hidden layer with 64 neurons and ReLU activation
    tf.keras.layers.Dense(10, activation='softmax')# Output layer with 10 neurons (for 10 classes)
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")


Epoch 1/5
Step 0, Loss: 2.3744, Accuracy: 0.0625
Step 100, Loss: 0.5307, Accuracy: 0.7259
Step 200, Loss: 0.6945, Accuracy: 0.7973
Step 300, Loss: 0.3799, Accuracy: 0.8291
Step 400, Loss: 0.1824, Accuracy: 0.8495
Step 500, Loss: 0.2466, Accuracy: 0.8632
Step 600, Loss: 0.0566, Accuracy: 0.8717
Step 700, Loss: 0.2934, Accuracy: 0.8791
Step 800, Loss: 0.3144, Accuracy: 0.8845
Step 900, Loss: 0.2282, Accuracy: 0.8883
Step 1000, Loss: 0.1397, Accuracy: 0.8923
Step 1100, Loss: 0.1488, Accuracy: 0.8955
Step 1200, Loss: 0.3757, Accuracy: 0.8987
Step 1300, Loss: 0.3101, Accuracy: 0.9016
Step 1400, Loss: 0.1963, Accuracy: 0.9038
Step 1500, Loss: 0.1458, Accuracy: 0.9060
Step 1600, Loss: 0.1349, Accuracy: 0.9083
Step 1700, Loss: 0.3246, Accuracy: 0.9110
Step 1800, Loss: 0.0778, Accuracy: 0.9131
Training Accuracy for epoch 1: 0.9144

Epoch 2/5
Step 0, Loss: 0.4738, Accuracy: 0.8750
Step 100, Loss: 0.0879, Accuracy: 0.9489
Step 200, Loss: 0.2489, Accuracy: 0.9510
Step 300, Loss: 0.0312, Accuracy: